<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Specify-imports-and-paths" data-toc-modified-id="Specify-imports-and-paths-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Specify imports and paths</a></span></li><li><span><a href="#Gather-training-data" data-toc-modified-id="Gather-training-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Gather training data</a></span><ul class="toc-item"><li><span><a href="#Define-MIDI-parsing-functions" data-toc-modified-id="Define-MIDI-parsing-functions-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Define MIDI parsing functions</a></span></li><li><span><a href="#Find-and-parse-midi-files" data-toc-modified-id="Find-and-parse-midi-files-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Find and parse midi files</a></span></li><li><span><a href="#Pickle-notes" data-toc-modified-id="Pickle-notes-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Pickle notes</a></span></li></ul></li><li><span><a href="#Prepare-training-data" data-toc-modified-id="Prepare-training-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Prepare training data</a></span><ul class="toc-item"><li><span><a href="#Define-data-preparation-functions" data-toc-modified-id="Define-data-preparation-functions-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Define data preparation functions</a></span></li><li><span><a href="#Create-note:int-mapping" data-toc-modified-id="Create-note:int-mapping-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Create note:int mapping</a></span></li><li><span><a href="#Create-input-sequences-and-corresponding-outputs" data-toc-modified-id="Create-input-sequences-and-corresponding-outputs-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Create input sequences and corresponding outputs</a></span></li><li><span><a href="#Transform-input-and-output" data-toc-modified-id="Transform-input-and-output-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Transform input and output</a></span></li></ul></li><li><span><a href="#Create-the-model" data-toc-modified-id="Create-the-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create the model</a></span></li><li><span><a href="#Train-the-model" data-toc-modified-id="Train-the-model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Train the model</a></span><ul class="toc-item"><li><span><a href="#Create-model-checkpoint" data-toc-modified-id="Create-model-checkpoint-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Create model checkpoint</a></span></li><li><span><a href="#Fit-the-model" data-toc-modified-id="Fit-the-model-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Fit the model</a></span></li></ul></li><li><span><a href="#Predict" data-toc-modified-id="Predict-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Predict</a></span><ul class="toc-item"><li><span><a href="#Define-predict-function" data-toc-modified-id="Define-predict-function-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Define predict function</a></span></li><li><span><a href="#Create-int:note-mapping" data-toc-modified-id="Create-int:note-mapping-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Create int:note mapping</a></span></li><li><span><a href="#Predict-notes-from-random-start-pattern" data-toc-modified-id="Predict-notes-from-random-start-pattern-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Predict notes from random start pattern</a></span></li></ul></li><li><span><a href="#Save-output" data-toc-modified-id="Save-output-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Save output</a></span><ul class="toc-item"><li><span><a href="#Define-MIDI-output-functions" data-toc-modified-id="Define-MIDI-output-functions-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Define MIDI output functions</a></span></li><li><span><a href="#Create-MIDI-file-from-predicted-output" data-toc-modified-id="Create-MIDI-file-from-predicted-output-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Create MIDI file from predicted output</a></span></li></ul></li></ul></div>

# MIDI Generation with Keras

The code below creates an LSTM RNN, and creates a model using various midi files as training data. The model generated is then used to produce some tunes... hopefully.

## Specify imports and paths

In [157]:
import pickle
from datetime import datetime
from pathlib import Path
from random import randint
from typing import Tuple

import numpy as np
from music21.chord import Chord
from music21.converter import parse
from music21.instrument import partitionByInstrument, Piano
from music21.note import Note
from music21.stream import Score, Stream
from music21.stream.iterator import RecursiveIterator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm


project_folder = Path('/home/tony/repos/rnn_midi_generator')
checkpoints_folder = project_folder / 'checkpoints'
data_folder = project_folder / 'data'
input_folder = project_folder / 'input'
output_folder = project_folder / 'output'

## Gather training data

### Define MIDI parsing functions

In [158]:
def get_notes_from_midi(midi: Score) -> RecursiveIterator:
    """
    Extracts notes from a midi file.
    """
    try:  # file has instrument parts
        partitioned_midi = partitionByInstrument(midi)
        return partitioned_midi.parts[0].recurse()
    except:  # file has notes in a flat structure
        return midi.flat.notes


def parse_notes(notes_to_parse: list) -> list:
    """
    Returns a list of the pitches of all the notes in the input.
    Chords are split into constituent notes and the pitches of each 
    is added to the output.
    """
    notes = []
    for element in notes_to_parse:
        if isinstance(element, Note):
            notes.append(str(element.pitch))
        elif isinstance(element, Chord):
            # notes.append('.'.join(str(n) for n in element.normalOrder))
            notes.extend([str(note.pitch) for note in element._notes])
    return notes

### Find and parse midi files

In [159]:
number_of_files_parsed = 0
notes = []

for file in input_folder.glob('*.mid'):
    midi = parse(file)
    notes_to_parse = get_notes_from_midi(midi)
    notes.extend(parse_notes(notes_to_parse))
    number_of_files_parsed += 1

print(f'Number of midi files parsed: {number_of_files_parsed}')

Number of midi files parsed: 17


### Pickle notes

In [160]:
with open(data_folder / 'notes', 'wb') as filepath:
    pickle.dump(notes, filepath)

## Prepare training data

### Define data preparation functions

In [24]:
def prepare_input_and_output(data_length: int, sequence_length: int,
                             data: list) -> Tuple[list, list]:
    """
    Creates two output lists whose items correspond to the sequence 
    which is used for prediction and the predicted next integer 
    respectively.
    """
    print(f'Data length: {data_length}')
    print(f'Sequence length: {sequence_length}')

    network_input = []
    network_output = []

    for i in range(data_length - sequence_length):
        sequence_in = data[i:i + sequence_length]
        sequence_out = data[i + sequence_length]
        network_input.append(sequence_in)
        network_output.append(sequence_out)

    return network_input, network_output


def reshape_and_normalize(input_data: list, num_possible_ints: int,
                          sequence_length: int) -> np.ndarray:
    """
    Reshapes and normalizes the input data according to the sequence
    length and the number of possible integer values.
    """
    reshaped = np.reshape(input_data, (len(input_data), sequence_length, 1))
    normalized_and_reshaped = reshaped / float(num_possible_ints)

    return normalized_and_reshaped

### Create note:int mapping
Create a dictionary mapping each note to an integer representing its position in an ordered list of all the notes.

In [83]:
sorted_possible_notes = sorted(set(notes))
note_to_int = dict((note, i) for i, note in enumerate(sorted_possible_notes))

### Create input sequences and corresponding outputs
The notes are mapped to their integer representation for the neural network to be able to process them.

In [100]:
sequence_length = 30
notes_length = len(notes)
notes_as_ints = [note_to_int[note] for note in notes]

network_input, network_output = prepare_input_and_output(
    data_length=notes_length,
    sequence_length=sequence_length,
    data=notes_as_ints,
)

Data length: 39846
Sequence length: 30


### Transform input and output
Reshape the `network_input` into a format compatible with LSTM layers and normalize. One-hot encode the `network_output`.

In [28]:
num_possible_notes = len(sorted_possible_notes)
print(f'Number of unique notes: {num_possible_notes}')

normalized_network_input = reshape_and_normalize(
    network_input, num_possible_notes, sequence_length)
network_output = to_categorical(network_output)

Number of unique notes: 70


## Create the model

Make a sequential model with several LSTM layers, 2 dense layers, ...

LSTM is used as the data is time-dependent.

In [30]:
model = Sequential()
model.add(LSTM(512,
               input_shape=(normalized_network_input.shape[1],
                            normalized_network_input.shape[2]),
               return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(num_possible_notes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 30, 512)           1052672   
_________________________________________________________________
dropout_3 (Dropout)          (None, 30, 512)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 512)           2099200   
_________________________________________________________________
dropout_4 (Dropout)          (None, 30, 512)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
__________

## Train the model

This might take a while...

### Create model checkpoint

In [164]:
timestamp = datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
model_name_template_string = timestamp + 'model_{epoch:02d}.hdf5'
checkpoint_filepath = checkpoints_folder / model_name_template_string

checkpoint = ModelCheckpoint(
    checkpoint_filepath.as_posix(),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min',
)

callbacks_list = [checkpoint]

### Fit the model

In [32]:
model.fit(
    normalized_network_input,
    network_output,
    batch_size=64,
    epochs=10,
    callbacks=callbacks_list,
)

Epoch 1/10
39816/39816 [==============================] - 1786s 45ms/step - loss: 3.7720
Epoch 2/10
39816/39816 [==============================] - 1764s 44ms/step - loss: 3.6208
Epoch 3/10
39816/39816 [==============================] - 1786s 45ms/step - loss: 3.5267
Epoch 4/10
39816/39816 [==============================] - 1784s 45ms/step - loss: 3.4116
Epoch 5/10
39816/39816 [==============================] - 2566s 64ms/step - loss: 3.2544
Epoch 6/10
39816/39816 [==============================] - 2227s 56ms/step - loss: 3.0563
Epoch 7/10
39816/39816 [==============================] - 2042s 51ms/step - loss: 2.8249
Epoch 8/10
39816/39816 [==============================] - 2082s 52ms/step - loss: 2.5681
Epoch 9/10
39816/39816 [==============================] - 1723s 43ms/step - loss: 2.3027
Epoch 10/10
39816/39816 [==============================] - 1718s 43ms/step - loss: 2.0364


## Predict

### Define predict function

In [150]:
def predict(model: Sequential, pattern: list, num_possible_values: int,
            num_predictions: int, output: list = []) -> list:
    """
    Recursively predict new values based on the given model and starting
    pattern.
    """
    if num_predictions == 0:
        return output

    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(num_possible_values)
    predicted_note = model.predict(prediction_input)
    note_as_int = np.argmax(predicted_note)
    output.append(note_as_int)
    pattern.append(note_as_int)
    pattern = pattern[1:]

    return predict(
        model, pattern, num_possible_values, num_predictions - 1, output,
    )

### Create int:note mapping

In [151]:
int_to_note = dict((v, k) for k, v in note_to_int.items())

### Predict notes from random start pattern

In [153]:
start = randint(0, len(network_input) - 1)
pattern = network_input[start]
num_predicted_notes = 100
prediction_output = predict(
    model,
    pattern,
    num_possible_notes,
    num_predicted_notes,
)

predicted_notes = [int_to_note[_int] for _int in prediction_output]
print(f'Number of notes predicted: {num_predicted_notes}')
print(f'First 20 predicted notes: {prediction_output[:20]}')

Number of notes predicted: 100
First 20 predicted notes: [61, 62, 61, 9, 9, 36, 36, 25, 10, 9, 9, 8, 8, 8, 8, 54, 56, 9, 10, 50]


## Save output 

### Define MIDI output functions

In [161]:
def convert_to_midi_note(note: str, offset: int) -> Note:
    """
    Convert a pitch string to a music21.note.Note object with the
    given offset. The note will be played by the piano.
    """
    new_note = Note(note)
    new_note.offset = offset
    new_note.storedInstrument = Trombone()

    return new_note

### Create MIDI file from predicted output

In [162]:
offset = 0
midi_notes = []

for note in predicted_notes:
    midi_notes.append(convert_to_midi_note(note, offset))
    offset += 0.5

timestamp = datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
midi_stream = Stream(midi_notes)
midi_stream.write('midi', fp=output_folder / f'{timestamp}.mid')

PosixPath('/home/tony/repos/rnn_midi_generator/output/2018-12-29_13:25:10.mid')